### Exemple de crawling et scraping du site web LaCentrale

In [2]:
import argparse, os, time
import urlparse, random
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import csv
import re
import json

#### Ouverture d'un Web Driver Chrome
Le web driver sera utilisé pour le crawl des pages du site web.

In [3]:
browser = webdriver.Chrome("E:/chromedriver.exe")

On détermine pour notre exemple la catégorie, la marque et l'année en tant que constantes.

In [4]:
categorie = "40" ## 40 refère à la catégorie Citadine
marque = "AUDI"
millesime = "2014"

On ouvre, avec le web driver, le lien du formulaire de recherche en donnant par requête GET la catégorie et marque pour voir les modèles disponibles.

In [5]:
link = "http://www.lacentrale.fr/Iframe_quot_auto.php?SS_CATEGORIE="+categorie+"&marque="+marque+"&modele=&millesime=&kindOfCustomer=vendeur"
browser.get(link)

#### Scraping des modèles de la marque AUDI, catégorie Citadine
Scraping simple des modèles de voiture disponible pour telles catégorie et marque.

In [6]:
page = BeautifulSoup(browser.page_source)
models_id = page.find(id="modele")

C:\Users\TONPC\Anaconda2\lib\site-packages\bs4\__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [7]:
print "Les modèle disponibles pour " + marque + " sont :"
i=0
for model in models_id.find_all('option'):
    if i==0:
        i=i+1
        continue
    print model["value"]
    

Les modèle disponibles pour AUDI sont :
7649880|A1
18705064|S1


On fixe le modèle qu'on veut et on valide le formulaire

In [8]:
modele = "7649880|A1"

In [9]:
link = "http://www.lacentrale.fr/Iframe_quot_auto.php?SS_CATEGORIE="+categorie+"&marque="+marque+"&modele="+modele+"&millesime="+millesime+"&kindOfCustomer=vendeur"
browser.get(link)

In [10]:
form = browser.find_element_by_css_selector('a.btnRed')
form.click()

#### Extraction de noms des voitures et liens vers leur cote
La fonction suivante extrait les noms des versions de toutes les voitures Audi A1 disponible ainsi que le lien vers la page de leur cote et stocke le tout dans un fichier .csv. 

In [12]:
def extract_car_links():
    page = BeautifulSoup(browser.page_source)
    listingCote = page.find('div', {'class':"listingCoteResult"})
    
    with open('links__cars.csv', 'wb') as f:
        writer = csv.writer(f)
        writer.writerow(["name","link"])
        links = page.find(id="results")
        cars_links= [[cars.find('h3').get_text(),cars.find('a')["href"]]for cars in listingCote.find_all('div', {'class':"listingCoteLine auto"})]
        writer.writerows(cars_links)
    pass

In [13]:
extract_car_links()

#### Extractions de la cote de la voiture avec kilométrage
La fonction suivante extrait la cote de la voiture selon un kilométrage déterminé depuis la page spécifique à la voiture puis constitue un objet json contenant les informations de la voitures (marque, modele, nom, caregorie, annee, cote, kilometrage) et le retourne.

In [47]:
def scrap_car(name,link):
    car = {}
    
    car['name'] =  name
    car['marque'] = marque
    car['annee'] = millesime
    car['categorie'] = categorie
    
    browser.get(link)
    page = BeautifulSoup(browser.page_source)
    
    cote_car = page.find('span',{'class':"clear lH20"}).get_text()
    car['modele'] = re.search('Audi (.+?) -', cote_car).group(1)
    
    cote_car = page.find('strong',{'class':"f24 bGrey9L txtRed pL15 mL15"}).get_text().strip().replace(" ", "")
    car['cote'] = re.findall(r'\d+', cote_car)[0]
    
    car_km = page.find('div',{'class':"f14 bold mT20 txtC"}).get_text().strip().replace(" ", "")
    car['car_km'] = re.findall(r'\d+', car_km)[0]
    
    #car_json = json.dumps(car, indent=4)
    
    return car

###### Exemple :

In [18]:
audi = scrap_car('(2) SPORTBACK 1.0 TFSI 95 ULTRA','http://www.lacentrale.fr/cote-auto-audi-a1-(2)+sportback+1.0+tfsi+95+ultra-2014.html')
print audi

{
    "categorie": "40", 
    "name": "(2) SPORTBACK 1.0 TFSI 95 ULTRA", 
    "car_km": "15000", 
    "marque": "AUDI", 
    "cote": "13900", 
    "annee": "2014", 
    "modele": "A1"
}


#### Connecion à la base de données MongoDb

In [38]:
from pymongo import MongoClient
Client = MongoClient()
db = Client["LaCentrale_DB"]
collection = db["cote_voitures"]

#### Extraction de toutes les versions de voitures de la marque Audi A1 et insertion dans la base de données

In [48]:
def load_links_scrap():
    cr = csv.reader(open("links__cars.csv","rb"))
    i=0
    for line in cr:
        print line[0]
        if i%10==0 and i!=0:
            print "Je suis arrive à " + str(i)
            time.sleep(60)
        car_info = scrap_car(line[0], "http://www.lacentrale.fr/" + line[1])
        #print car_info
        collection.insert(car_info)
        #if i==0: break
        print "Voiture "+ str(i) + " a bien été extraite !"
        i=i+1
    print "Succés!"

In [49]:
load_links_scrap()

(2) SPORTBACK 1.0 TFSI 95 ULTRA
Voiture 0 a bien été extraite !
(2) SPORTBACK 1.0 TFSI 95 ULTRA AMBIENTE
Voiture 1 a bien été extraite !
(2) SPORTBACK 1.0 TFSI 95 ULTRA AMBIENTE S TRONIC
Voiture 2 a bien été extraite !
(2) SPORTBACK 1.0 TFSI 95 ULTRA AMBITION
Voiture 3 a bien été extraite !
(2) SPORTBACK 1.0 TFSI 95 ULTRA AMBITION LUXE
Voiture 4 a bien été extraite !
(2) SPORTBACK 1.0 TFSI 95 ULTRA AMBITION LUXE S TRONIC
Voiture 5 a bien été extraite !
(2) SPORTBACK 1.0 TFSI 95 ULTRA AMBITION S TRONIC
Voiture 6 a bien été extraite !
(2) SPORTBACK 1.0 TFSI 95 ULTRA BUSINESS LINE
Voiture 7 a bien été extraite !
(2) SPORTBACK 1.0 TFSI 95 ULTRA BUSINESS LINE S TRONIC
Voiture 8 a bien été extraite !
(2) SPORTBACK 1.0 TFSI 95 ULTRA S LINE
Voiture 9 a bien été extraite !
(2) SPORTBACK 1.0 TFSI 95 ULTRA S LINE S TRONIC
Je suis arrive à 10
Voiture 10 a bien été extraite !
(2) SPORTBACK 1.0 TFSI 95 ULTRA S TRONIC
Voiture 11 a bien été extraite !
(2) SPORTBACK 1.4 TDI ULTRA 90
Voiture 12 a bien é

C:\Users\TONPC\Anaconda2\lib\site-packages\ipykernel\__main__.py:11: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.


###### On s'assure que la base de données est bien rempli

In [53]:
print "Le nombre de voitures stockés : " + str(collection.count())

Le nombre de voitures stockés : 169
